In [ ]:
from flask import Flask, jsonify
from flask_cors import CORS  # Import CORS

import socket
import struct
import time
import ctypes
import mmap
from threading import Thread

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Define telemetry data structures
class vec3(ctypes.Structure):
    _pack_ = 4
    _fields_ = [
        ('x', ctypes.c_float),
        ('y', ctypes.c_float),
        ('z', ctypes.c_float)
    ]

class acsVehicleInfo(ctypes.Structure):
    _pack_ = 4
    _fields_ = [
        ('carId', ctypes.c_int32),
        ('driverName', ctypes.c_char * 64),
        ('carModel', ctypes.c_char * 64),
        ('speedMS', ctypes.c_float),
        ('bestLapMS', ctypes.c_int32),
        ('lapCount', ctypes.c_int32),
        ('currentLapInvalid', ctypes.c_int32),
        ('currentLapTimeMS', ctypes.c_int32),
        ('lastLapTimeMS', ctypes.c_int32),
        ('worldPosition', vec3),
        ('isCarInPitline', ctypes.c_int32),
        ('isCarInPit', ctypes.c_int32),
        ('carLeaderboardPosition', ctypes.c_int32),
        ('carRealTimeLeaderboardPosition', ctypes.c_int32),
        ('spLineLength', ctypes.c_float),
        ('isConnected', ctypes.c_int32),
        ('suspensionDamage', ctypes.c_float * 4),
        ('engineLifeLeft', ctypes.c_float),
        ('tyreInflation', ctypes.c_float * 4)
    ]

class SPageFileSimHub(ctypes.Structure):
    _pack_ = 4
    _fields_ = [
        ('numVehicles', ctypes.c_int32),
        ('focusVehicle', ctypes.c_int32),
        ('serverName', ctypes.c_char * 512),
        ('vehicleInfo', acsVehicleInfo * 64),
        ('acInstallPath', ctypes.c_char * 512),
        ('isInternalMemoryModuleLoaded', ctypes.c_int32),
        ('pluginVersion', ctypes.c_char * 32)
    ]

class SimHubShared:
    def __init__(self):
        self._acpmf_simhub = mmap.mmap(0, ctypes.sizeof(SPageFileSimHub), "acpmf_simhub")
        self.simhub = SPageFileSimHub.from_buffer(self._acpmf_simhub)

# Global variable to store telemetry data
telemetry_data = []

class ACUDP:
    def __init__(self, ip, port=9996):
        self.udp_address = ip
        self.udp_port = port
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.sock.settimeout(5)
        print(f"Socket initialized with IP {ip} and port {port}")

        # Handshake data
        self.hand = struct.pack('iii', 1, 1, 0)
        self.update = struct.pack('iii', 1, 1, 1)
        self.spot = struct.pack('iii', 1, 1, 2)
        self.quit = struct.pack('iii', 0, 0, 0)
        self.is_update = False

        # Initialize SimHub shared memory
        try:
            self.simhub_shared = SimHubShared()
            print("SimHub shared memory initialized.")
        except Exception as e:
            print(f"Error initializing SimHub shared memory: {e}")
            self.simhub_shared = None

    def send_packet(self, data):
        try:
            self.sock.sendto(data, (self.udp_address, self.udp_port))
            print(f"Packet sent to {self.udp_address}:{self.udp_port}")
        except Exception as e:
            print(f"Failed to send packet: {e}")

    def send_handshake(self):
        print("Sending handshake...")
        self.send_packet(self.hand)

    def send_update(self):
        print("Sending update...")
        self.send_packet(self.update)
        self.is_update = True

    def send_spot(self):
        print("Sending spot...")
        self.send_packet(self.spot)
        self.is_update = False

    def send_quit(self):
        print("Sending quit...")
        self.send_packet(self.quit)

    def read_update(self):
        try:
            buffer, _ = self.sock.recvfrom(1024)
            print(f"Received buffer length: {len(buffer)}")
            # Process and print buffer data as needed
            return buffer
        except socket.timeout:
            print("Timeout: No data received")
        except Exception as e:
            print(f"Error receiving data: {e}")

    def read_spot(self):
        # Placeholder for spot data reading
        print("Reading spot data...")

    def read_simhub_data(self):
        """Read all available data from SimHub shared memory for each vehicle."""
        global telemetry_data
        telemetry_data = []  # Clear previous data

        if self.simhub_shared is None:
            print("SimHub shared memory not available.")
            return None
        try:
            simhub_data = self.simhub_shared.simhub
            for i in range(simhub_data.numVehicles):
                vehicle = simhub_data.vehicleInfo[i]
                telemetry_data.append({
                    "CarID": vehicle.carId,
                    "DriverName": vehicle.driverName.decode('utf-8').strip(),
                    "CarModel": vehicle.carModel.decode('utf-8').strip(),
                    "SpeedKmh": vehicle.speedMS * 3.6,
                    "BestLapTimeSec": vehicle.bestLapMS / 1000,
                    "LapCount": vehicle.lapCount,
                    "CurrentLapTimeSec": vehicle.currentLapTimeMS / 1000,
                    "LastLapTimeSec": vehicle.lastLapTimeMS / 1000,
                    "Position": {
                        "x": vehicle.worldPosition.x,
                        "y": vehicle.worldPosition.y,
                        "z": vehicle.worldPosition.z
                    },
                    "InPit": bool(vehicle.isCarInPit),
                    "LeaderboardPosition": vehicle.carLeaderboardPosition,
                    "SuspensionDamage": list(vehicle.suspensionDamage),
                    "EngineLifeLeft": vehicle.engineLifeLeft,
                    "TyreInflation": list(vehicle.tyreInflation)
                })
            return simhub_data
        except Exception as e:
            print(f"Error reading SimHub data: {e}")
            return None

    def read(self):
        if self.is_update:
            self.read_update()
        else:
            self.read_spot()
        self.read_simhub_data()  # Include SimHub data read

# Flask endpoint for JSON data
@app.route('/telemetry')
def get_telemetry():
    return jsonify(telemetry_data)

def start_flask():
    app.run(host="0.0.0.0", port=5000)

if __name__ == "__main__":
    # Start Flask server in a separate thread
    flask_thread = Thread(target=start_flask)
    flask_thread.daemon = True
    flask_thread.start()

    # Start reading telemetry data
    acudp = ACUDP("127.0.0.1", 9996)
    while True:
        acudp.send_handshake()
        acudp.send_update()
        acudp.read()
        time.sleep(1)


Socket initialized with IP 127.0.0.1 and port 9996
SimHub shared memory initialized.
Sending handshake...
Packet sent to 127.0.0.1:9996
Sending update...
Packet sent to 127.0.0.1:9996
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.202:5000
Press CTRL+C to quit


Received buffer length: 408


127.0.0.1 - - [04/Nov/2024 11:53:32] "GET /telemetry HTTP/1.1" 200 -


Sending handshake...
Packet sent to 127.0.0.1:9996
Sending update...
Packet sent to 127.0.0.1:9996
Received buffer length: 328


...

127.0.0.1 - - [04/Nov/2024 12:13:48] "GET /telemetry HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2024 12:13:48] "GET /telemetry HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2024 12:13:50] "GET /telemetry HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2024 12:13:50] "GET /telemetry HTTP/1.1" 200 -


Sending handshake...
Packet sent to 127.0.0.1:9996
Sending update...
Packet sent to 127.0.0.1:9996
Received buffer length: 408


127.0.0.1 - - [04/Nov/2024 12:13:52] "GET /telemetry HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2024 12:13:52] "GET /telemetry HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2024 12:13:54] "GET /telemetry HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2024 12:13:54] "GET /telemetry HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2024 12:13:56] "GET /telemetry HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2024 12:13:56] "GET /telemetry HTTP/1.1" 200 -


Sending handshake...
Packet sent to 127.0.0.1:9996
Sending update...
Packet sent to 127.0.0.1:9996
Received buffer length: 408


127.0.0.1 - - [04/Nov/2024 12:13:58] "GET /telemetry HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2024 12:13:58] "GET /telemetry HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2024 12:14:00] "GET /telemetry HTTP/1.1" 200 -
